# What this notebook teaches

1. Advanced `Masking`: combining multiple masks
2. `Apply` operation: for **transforming** columns and **creating** new ones
3. `Groupby` operation
4. More `plotting` functionalities

# Options

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Imports

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Read the data

In [ ]:
local_path = os.path.join('data/global-data-on-sustainable-energy.csv')
url = "https://raw.githubusercontent.com/samsung-ai-course/6-7-edition/refs/heads/main/Data%20Wrangling/Data%20Wrangling%20-%20Pandas-Advanced/Data%20Analysis%20Advanced/data/global-data-on-sustainable-energy.csv"
df = pd.read_csv(url)
df.head(3)
df.shape

# Advanced Masking

![](media/dino.jpeg)

## Using `&` (AND) operator

Let's say i want:
1. `United States`
2. from `2012`


In [ ]:
mask_us = df['Entity'] == 'United States'
mask_2012 = df['Year'] == 2012

In [ ]:
df.loc[mask_us & mask_2012]

And if i now want all countries where:
1. In the year `2018`
2. had `gdp_growth` of at least 7.5

In [ ]:
mask_2018 = df['Year'] == 2018
mask_gdp_growth = df['gdp_growth'] > 7.5

In [ ]:
df.loc[mask_gdp_growth & mask_2018,:]

If we just want the list:

In [ ]:
df.loc[mask_gdp_growth & mask_2018,'Entity'].tolist()

And all the countries whose gdp went down in 2018?

In [ ]:
mask_2018 = df['Year'] == 2018
mask_negative_gdp_growth = df['gdp_growth'] < 0

df.loc[mask_negative_gdp_growth & mask_2018,'Entity'].tolist()

## Using `|` (OR) operator

I want all data from both:
1. `Albania`
2. and `Angola`

In [ ]:
mask_albania = df['Entity'] == 'Albania'
mask_angola = df['Entity'] == 'Angola'

In [ ]:
df.loc[mask_albania | mask_angola,:].head(3) # remove the .head to confirm the results

Pretty cool! However, we don't have to stick to conditions on the same columns:

For `2020`, I want the countries where `either`:
1. The `GDP` grew by at least 4.0
2. The `Primary energy consumption per capita` was above 1050

In [ ]:
mask_2020 = df['Year'] == 2020
mask_gdp = df['gdp_growth'] > 4.0
mask_energy_consumption = df['Primary energy consumption per capita (kWh/person)'] > 1050

In [ ]:
df.loc[mask_2020 & (mask_gdp | mask_energy_consumption)]

Very cool! Now i can compute any analysis for this data

In [ ]:
# what is the distribution of land area density of these countries?

df.loc[mask_2020 & (mask_gdp | mask_energy_consumption),'Land Area(Km2)'].describe().astype(int)

In [ ]:
# What is the average `Financial aid to developing countries` of these countries in 2020, in thousands of $

df.loc[mask_2020 & (mask_gdp | mask_energy_consumption),'Financial flows to developing countries (US $)'].mean() / 1_000

# `Apply` operation

Let's go back to the supermarket sales

In [ ]:
local_path = os.path.join('data','supermarket_sales.csv')
url = "https://raw.githubusercontent.com/samsung-ai-course/6-7-edition/refs/heads/main/Data%20Wrangling/Data%20Wrangling%20-%20Pandas-Advanced/Data%20Analysis%20Advanced/data/supermarket_sales.csv"
df = pd.read_csv(url)
df.head(3)
df.shape

## In strings

`.apply` can be incredibly useful for string columns to help you parse and clean them for further analysis.

For example, it would have been useful to use the Date column to at least analyze the data throughout time.

We know that this dataset is only for the year 2019, so let's extract the month and day of the month so that we can use this information for further analysis

In [ ]:
df['Date'].apply(lambda row: row.split('/')[0])

We applied a function to each row, which extracts the first string after splitting by the character `/` in order to get the month part of the date

In [ ]:
#doing the same to extract the day
df['Date'].apply(lambda row: row.split('/')[1]) # day of the month

We can create new columns in our dataset with this info! Let's add this new data to our dataset

In [ ]:
df['month'] = df['Date'].apply(lambda row: row.split('/')[0])
df['day'] = df['Date'].apply(lambda row: row.split('/')[1])

In [ ]:
df.head(3)

Now that we have this extra information, we can do a lot more analysis to the data!

In [ ]:
# how many sales in January?
mask_january = df['month'] == "1"

df.loc[mask_january,'Total'].sum()

In [ ]:
# what was the day of february with the largest purchase?
mask_february = df['month'] == "2"
index = df.loc[mask_february,'Total'].idxmax()

df.loc[index,'day']

In [ ]:
# and the rest of the data for this day?
df.loc[index,:].to_frame('Value')

In [ ]:
df.head(3)

## In numerical data

There are some columns that are computed through operations on other columns:

For example:

1. `Tax 5%`: is simply `Unit price` * `Quantity` * 0.05
2. `Total`: is `Unit price` * `Quantity` + `Tax 5%`


Let's do the `Tax 5%`

In [ ]:
total_before_taxes = df['Unit price'] * df['Quantity']
total_before_taxes

In [ ]:
tax_5 = total_before_taxes.apply(lambda row: row * 0.05)
tax_5

How do we compare our solution with the `Tax 5%` column? As always there are many ways to do that:

In [ ]:
# visually
pd.concat([tax_5.rename('Ours'), df['Tax 5%']],axis=1)

In [ ]:
# numerically 
(tax_5 - df['Tax 5%']).round(5).value_counts()

Awesome! This confirms we did it well

> Let's say that now we want the `Total` in thousands of $ 

In [ ]:
df.loc[:,'Total (thousands $)'] = df['Total'].apply(lambda row: row / 1000)

In [ ]:
df.head(5)

# Groupby


[complete docs for groupby in Pandas](https://pandas.pydata.org/docs/reference/groupby.html)


Often you want to compute information across all unique categories of a column.

In [ ]:
local_path = os.path.join('data','global-data-on-sustainable-energy.csv')
url = "https://raw.githubusercontent.com/samsung-ai-course/6-7-edition/refs/heads/main/Data%20Wrangling/Data%20Wrangling%20-%20Pandas-Advanced/Data%20Analysis%20Advanced/data/global-data-on-sustainable-energy.csv"
df = pd.read_csv(url)
df.head(3)
df.shape

> What was the average gdp growth across **all countries** between `2015` and `2019`?

In [ ]:
mask_years = (df['Year'] >= 2015) & (df['Year'] <= 2019)

(df
 .loc[mask_years,:] # select only data for the years between 2015 and 2019
 .groupby('Entity') # group by the countries
 ['gdp_growth'].mean() # get the average value of gdp_growth for each country
 .sort_values(ascending=False) # sort the result in descending order
)

## Plotting

pretty cool right? The results of groupby make some very good candidates for visualization

In [ ]:
(df
 .loc[mask_years,:] # select only data for the years between 2015 and 2019
 .groupby('Entity') # group by the countries
 ['gdp_growth'].mean() # get the average value of gdp_growth for each country
 .sort_values(ascending=False) # sort the result in descending order
 .plot.bar()
)

![](media/ugly.jpg)

**Ouch** that hurts. There are so many countries in this dataset that they don't fit very well in the figure

We can try a few things:
1. Increase the figure size
2. rotate the labels by 45 degrees
3. plot only the top 10 

In [ ]:
(df
 .loc[mask_years,:] # select only data for the years between 2015 and 2019
 .groupby('Entity') # group by the countries
 ['gdp_growth'].mean() # get the average value of gdp_growth for each country
 .sort_values(ascending=False) # sort the result in descending order
 .iloc[:10] # select top 10
 .plot.bar(rot=45,
           figsize=(16,8),
           title='Top 10 Countries by gdp growth between 2015 and 2019', 
           ylabel='% gdp growth',
           xlabel='Country'
          )
)

Much better! Font size for the labels and title are a bit small though, let's increase them

In [ ]:
ax = (df
     .loc[mask_years,:] # select only data for the years between 2015 and 2019
     .groupby('Entity') # group by the countries
     ['gdp_growth'].mean() # get the average value of gdp_growth for each country
     .sort_values(ascending=False) # sort the result in descending order
     .iloc[:10] # select top 10
     .plot.bar(rot=45,
               figsize=(16,8)
              )
    )

ax.set_xlabel('Country', fontsize=20);
ax.set_ylabel('% gdp growth', fontsize=20);
ax.set_title('Top 10 Countries by gdp growth between 2015 and 2019', fontsize=20);

## Continuing Groupby

Groupby is a very powerful tool, let's analyze more use cases we can do with it

> What is the average gdp growth by year of our list of countries?

In [ ]:
df.groupby('Year')['gdp_growth'].mean()

> In total, how much did each country financially aid developing countries? (in billions of \$)

In [ ]:
df.groupby('Entity')['Financial flows to developing countries (US $)'].sum().sort_values(ascending=False) / 1_000_000_000

again, we can make a visualization out of this

In [ ]:
result = df.groupby('Entity')['Financial flows to developing countries (US $)'].sum().sort_values(ascending=False) / 1_000_000_000

result.head(10).plot.bar(title='Top countries with most financial aid to developing countries between 2000 and 2020',rot=45,xlabel='Country',ylabel='$ (billions)')

> What was the worst GDP Growth for each country

In [ ]:
df.groupby('Entity')['gdp_growth'].min().sort_values()

Where does portugal stand in this?

In [ ]:
df.groupby('Entity')['gdp_growth'].min()['Portugal']

Also, sometimes it can be very useful to know the size of each group. For example:
> How many records do we have per country

In [ ]:
df.groupby('Entity').size().sort_values()

## Creating your own custom groupby operations with `.apply`

Often, you will need to perform a custom operation on each group.

For example: 
> Give me the countries with the highest gdp growth **by year**

to solve this, we would need:
1. To group the data by year
2. In each group find the index of largest `gdp growth`
3. Use that index to find the `Country` of largest growth
4. repeat for all groups (though groupby handles that for us)

This means we need data from two columns in our groupby operation: `gdp_growth` and `Entity`

In [ ]:
# Give me the countries with the highest gdp growth **by year**

df.groupby('Year').apply(lambda subdf: subdf.loc[subdf['gdp_growth'].idxmax(),'Entity'])

pretty awesome right?

We can keep going and perform an even more complex query:
> What is the country that had the highest gdp growth most times?

In [ ]:
(df
 .groupby('Year')
 .apply(lambda subdf: subdf.loc[subdf['gdp_growth'].idxmax(),'Entity']) # This is what we had already
 .value_counts()
 .where(lambda x: x == x.max())
 .dropna()
 .index
 .tolist()
)

![](media/cat.jpeg)